In [1]:
from IPython import get_ipython
ipython = get_ipython()
if ipython is not None:
    ipython.magic("%load_ext autoreload")
    ipython.magic("%autoreload 2")

import os
import sys
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('..')
import torch
import re

import acdc
from utils.prune_utils import get_3_caches, split_layers_and_heads
from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.acdc_utils import TorchIndex, EdgeType
import numpy as np
import torch as t
from torch import Tensor
import einops
import itertools

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly
from rich import print as rprint
from rich.table import Table

from jaxtyping import Float, Bool
from typing import Callable, Tuple, Union, Dict, Optional

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

/tmp/ipykernel_8522/1027636265.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%load_ext autoreload")
/tmp/ipykernel_8522/1027636265.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%autoreload 2")


Device: cuda


# Model Setup

In [2]:
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# Dataset Setup

In [3]:
from ioi_dataset import IOIDataset, format_prompt, make_table
N = 25
clean_dataset = IOIDataset(
    prompt_type='mixed',
    N=N,
    tokenizer=model.tokenizer,
    prepend_bos=False,
    seed=1,
    device=device
)
corr_dataset = clean_dataset.gen_flipped_prompts('ABC->XYZ, BAB->XYZ')

make_table(
  colnames = ["IOI prompt", "IOI subj", "IOI indirect obj", "ABC prompt"],
  cols = [
    map(format_prompt, clean_dataset.sentences),
    model.to_string(clean_dataset.s_tokenIDs).split(),
    model.to_string(clean_dataset.io_tokenIDs).split(),
    map(format_prompt, clean_dataset.sentences),
  ],
  title = "Sentences from IOI vs ABC distribution",
)

                                      Sentences from IOI vs ABC distribution                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ IOI prompt                              ┃ IOI subj ┃ IOI indirect obj ┃ ABC prompt                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ When Victoria and Jane got a snack at   │ Jane     │ Victoria         │ When Victoria and Jane got a snack at   │
│ the store, Jane decided to give it to   │          │                  │ the store, Jane decided to give it to   │
│ Victoria                                │          │                  │ Victoria                                │
│                                         │          │                  │                                         │
│ When Sullivan and Rose got a necklace   │ Sullivan │ Rose             │ When Sullivan and Rose got a necklace   │
│ at the garden, Sullivan decided to give │          │                  │ at the garden, Sullivan decided to give │
│ it to Rose                              │          │                  │ it to Rose                              │
│                                         │          │                  │                                         │
│ When Alan and Alex got a drink at the   │ Alex     │ Alan             │ When Alan and Alex got a drink at the   │
│ store, Alex decided to give it to Alan  │          │                  │ store, Alex decided to give it to Alan  │
│                                         │          │                  │                                         │
│ Then, Jessica and Crystal had a long    │ Jessica  │ Crystal          │ Then, Jessica and Crystal had a long    │
│ argument, and afterwards Jessica said   │          │                  │ argument, and afterwards Jessica said   │
│ to Crystal                              │          │                  │ to Crystal                              │
│                                         │          │                  │                                         │
│ Then, Jonathan and Kevin were working   │ Kevin    │ Jonathan         │ Then, Jonathan and Kevin were working   │
│ at the school. Kevin decided to give a  │          │                  │ at the school. Kevin decided to give a  │
│ necklace to Jonathan                    │          │                  │ necklace to Jonathan                    │
│                                         │          │                  │                                         │
└─────────────────────────────────────────┴──────────┴──────────────────┴─────────────────────────────────────────┘

# Metric Setup

In [4]:
def ave_logit_diff(
    logits: Float[Tensor, 'batch seq d_vocab'],
    ioi_dataset: IOIDataset,
    per_prompt: bool = False
):
    '''
        Return average logit difference between correct and incorrect answers
    '''
    # Get logits for indirect objects
    io_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.io_tokenIDs]
    s_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.s_tokenIDs]
    # Get logits for subject
    logit_diff = io_logits - s_logits
    return logit_diff if per_prompt else logit_diff.mean()

with t.no_grad():
    clean_logits = model(clean_dataset.toks)
    corrupt_logits = model(corr_dataset.toks)
    clean_logit_diff = ave_logit_diff(clean_logits, clean_dataset).item()
    corrupt_logit_diff = ave_logit_diff(corrupt_logits, corr_dataset).item()

def ioi_metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    ioi_dataset: IOIDataset = clean_dataset
 ):
    patched_logit_diff = ave_logit_diff(logits, ioi_dataset)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

def negative_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -ioi_metric(logits)
    
# Get clean and corrupt logit differences
with t.no_grad():
    clean_metric = ioi_metric(clean_logits, corrupt_logit_diff, clean_logit_diff, clean_dataset)
    corrupt_metric = ioi_metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, corr_dataset)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

Clean direction: 2.805180072784424, Corrupt direction: 1.2352441549301147
Clean metric: 1.0, Corrupt metric: 0.0


# Run Experiment

In [5]:
# get the 2 fwd and 1 bwd caches; cache "normalized" and "result" of attn layers
clean_cache, corrupted_cache, clean_grad_cache = get_3_caches(
    model, 
    clean_dataset.toks,
    corr_dataset.toks,
    metric=negative_ioi_metric,
    pass_tokens_to_metric=False, # TODO
    mode = "edge",
)

In [6]:
clean_head_act = split_layers_and_heads(clean_cache.stack_head_results(), model=model)
corr_head_act = split_layers_and_heads(corrupted_cache.stack_head_results(), model=model)

In [7]:
stacked_grad_act = torch.zeros(
    3, # QKV
    model.cfg.n_layers,
    model.cfg.n_heads,
    clean_head_act.shape[-3], # Batch
    clean_head_act.shape[-2], # Seq
    clean_head_act.shape[-1], # D
)

for letter_idx, letter in enumerate("qkv"):
    for layer_idx in range(model.cfg.n_layers):
        stacked_grad_act[letter_idx, layer_idx] = einops.rearrange(clean_grad_cache[f"blocks.{layer_idx}.hook_{letter}_input"], "batch seq n_heads d -> n_heads batch seq d")

In [10]:
results = {}

for upstream_layer_idx in range(model.cfg.n_layers):
    for upstream_head_idx in range(model.cfg.n_heads):
        for downstream_letter_idx, downstream_letter in enumerate("qkv"):
            for downstream_layer_idx in range(upstream_layer_idx+1, model.cfg.n_layers):
                for downstream_head_idx in range(model.cfg.n_heads):
                    results[
                        (
                            upstream_layer_idx,
                            upstream_head_idx,
                            downstream_letter,
                            downstream_layer_idx,
                            downstream_head_idx,
                        )
                    ] = (stacked_grad_act[downstream_letter_idx, downstream_layer_idx, downstream_head_idx].cpu() * (clean_head_act[upstream_layer_idx, upstream_head_idx] - corr_head_act[upstream_layer_idx, upstream_head_idx]).cpu()).sum()

In [11]:
sorted_results = sorted(results.items(), key=lambda x: x[1].abs(), reverse=True)

In [15]:
print("Top 10 most important edges:")
for i in range(10):
    print(
        f"{sorted_results[i][0][0]}:{sorted_results[i][0][1]} -> {sorted_results[i][0][3]}:{sorted_results[i][0][4]}",
    )

Top 10 most important edges:
9:9 -> 11:10
10:7 -> 11:10
5:5 -> 8:6
9:9 -> 10:7
5:5 -> 6:9
9:6 -> 11:10
4:11 -> 6:9
9:6 -> 10:7
5:5 -> 7:9
10:10 -> 11:10
